<a href="https://colab.research.google.com/github/ApeWizard/mat281-entregable/blob/main/docs/codes/homeworks/Tarea%201%20Aplica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problema 1:
-Se importan librerias
-Se usa try en caso de que hayan errores con el path que pone el usuario

In [ ]:
import numpy as np
from PIL import Image


#Problema 1:
try:
    gatito = Image.open("images/gatito.png")
    gatito_np = np.array(gatito)

    secret_list = []

    for canal in range(gatito_np.shape[2]):
        secret_aux = gatito_np[:, :, canal] % 2
        secret_aux = secret_aux * 255
        secret_aux = secret_aux.astype(np.uint8)
        secret_list.append(secret_aux)

    print(f"la lista secreta tiene {len(secret_list)} elementos")

    secret_np = np.concatenate(secret_list, axis=1)
    print(f"shape de la imagen secreta: {secret_np.shape}")

    secret_img = Image.fromarray(secret_np, mode="L")
    secret_img.save("images/secret_revealed.png")
    print("Imagen secreta guardada")

except FileNotFoundError as e:
    print(f"Warning: No se encontro gatito.png: {e}")

# Problema 2:
try:
    my_img = Image.open("images/gatito.png")

    my_img_np = np.array(my_img.convert('L'))
    print(f"shape de imagen: {my_img_np.shape}")

    umbral = 20
    my_img_np_aux = (my_img_np > umbral).astype(np.uint8)

    my_img_split = np.array_split(my_img_np_aux, 3, axis=1)

    cat = Image.open("images/gatito_original.png")
    cat_np = np.array(cat).copy()  #asegura escritura
    print(f"shape de imagen cargada: {cat_np.shape}")


    mask = cat_np % 2 == 1
    cat_np[mask] -= 1

    for channel in range(3):
        cat_np[:, :, channel] += my_img_split[channel]

    cat_secret_im = Image.fromarray(cat_np)
    cat_secret_im.save("images/my_secret.png")
    print("Secreto guardado!")

except FileNotFoundError as e:
    print(f"Warning: No se pudo procesar la imagen para esconder: {e}")

def imagenception(filepath):
    try:
        img = Image.open(filepath)
        img_np = np.array(img)
        secret_list = []
        for i in range(img_np.shape[2]):
            secret_aux = (img_np[:, :, i] % 2) * 255
            secret_list.append(secret_aux.astype(np.uint8))
        secret_np = np.concatenate(secret_list, axis=1)
        return Image.fromarray(secret_np, mode="L")
    except Exception as e:
        print(f"Error con imagen: {e}")
        return None

# Test
try:
    my_secret_img = imagenception("images/my_secret.png")
    if my_secret_img:
        my_secret_img.save("images/decoded_secret.png")
        print("Decoding function tested successfully!")
except Exception as e:
    print(f"Could not test decoding: {e}")

Arriba esta el probema 1. Notar que los directorios varian segun computador y sistema operativo (Linux/Windows/Mac) entonces es necesario cambiar el path de las imagenes. Tanto donde se guardan como de donde se extraen. Esto aplica para los 3 problemas.

Problema 2: Si yo descargo los datos, se guardaran en un lugar especifico de mi escritorio, y no es apto para el usuario. Por esto, se asume que el usuario tiene los datos y debe usar su propio path para accederlos.

In [ ]:
import numpy as np
import pandas as pd



try:
    df_dict = {}
    for year in [2015, 2016, 2017]:
        try:
            df = pd.read_csv(f"data/world-happiness/{year}.csv")
            df_dict[year] = df.assign(Year=year)
        except FileNotFoundError:
            print(f"Warning: No existen datos para {year}")

    if not df_dict:
        raise ValueError("No se encontraron los datos")

    intersection_columns = reduce(np.intersect1d,
                                [df_i.columns.values for df_i in df_dict.values()]).tolist()
    print(f"Common columns: {intersection_columns}")
    #se usa reduce para conseguir las columnas columnas en caso de que difieran.


    def clean_column_name(x):
        x = x.lower().replace(' ', '_')
        x = re.sub(r'\([^)]*\)', '', x)
        x = x.strip('_')
        return x
    #Se limpian los nombres de columnas en caso de que no sean iguales.
    happiness = (pd.concat(df_dict.values()).reset_index(drop=True)[intersection_columns].rename(columns=clean_column_name))

    print(f"dataframe shape: {happiness.shape}")
    print(f"dataframe columns: {happiness.columns}")
    country_counts = happiness['country'].value_counts()
    missing_all_years = country_counts[country_counts < 3]
    print(f"Countries missing measurements: {len(missing_all_years)}")

    bad_country_names_dict = {
        "Hong Kong S.A.R., China": "Hong Kong",
        "Taiwan Province of China": "Taiwan",
        "Somaliland region": "Somaliland Region"
    }
    happiness = happiness.assign(
        country=happiness['country'].replace(bad_country_names_dict)
    )
    #Aca se podria usar .map pero no se si es parte del curso

    pivot_happiness = happiness.pivot(
        index='year',
        columns='country',
        values='happiness_score'
    ).fillna("") #Se rellenan los valores NULL con ""

    mean_happiness = happiness.groupby('country')['happiness_score'].mean().nlargest(3)
    print(f"Top 3 paises mas felices:\n{mean_happiness}")


    factor_cols = [col for col in happiness.columns
                   if col not in ['country', 'year', 'happiness_score', 'happiness_rank']
                   and pd.api.types.is_numeric_dtype(happiness[col])]

    if factor_cols:
        hap_mean_factors = happiness.groupby('year')[factor_cols].mean()
        print(f"Factor con contribucion mas alta por a\~no :\n{hap_mean_factors.idxmax(axis=1)}")
    else:
        print("No numeric factor columns found")

except Exception as e:
    print(f"Error procesando los datos {e}")

try:
    suicide = pd.read_csv("data/suicide_rates.csv")

    suicides_agg = (
        suicide.groupby(['country', 'year'])
        .agg({
            'population': 'sum',
            'suicides_no': 'sum'
        })
        .reset_index()
        .assign(
            suicides_ratio_100k=lambda x: (x['suicides_no'] / x['population']) * 100000,
            suicides_rank=lambda x: x.groupby('year')['suicides_ratio_100k']
                                    .rank(method='dense', ascending=False)
        )
    )
    #No se recomienda usar lambda porque es lento bases de datos muy grandes. Mas
    #eficiente seria un metodo vectorizado con numpy o mejor aun numpy (especialmente ya que
    #son numeros). Si se trata de valores string se suele hacer un "embedding".
    if 'happiness' in locals():
        hap_sui = happiness.merge(suicides_agg, on=['country', 'year'], how='inner')
        #Merge puede duplicar valores si existen columnas iguales en los dataframes,
        #Se suele borrar el que tiene prefijo "_y".
        if 'happiness_rank' in hap_sui.columns and 'suicides_rank' in hap_sui.columns:
            corr_ranks = hap_sui[['happiness_rank', 'suicides_rank']].corr().iloc[0, 1]
            print(f"Correlacion: {corr_ranks:.3f}") #3 puntos decimales

            # Correlacion por a\~no:
            corr_by_year = hap_sui.groupby('year').apply(
                lambda x: x[['happiness_rank', 'suicides_rank']].corr().iloc[0, 1]
            )
            print(f"Correlacion por a\~no :\n{corr_by_year}")

except FileNotFoundError:
    print("Warning: suicide_rates.csv no se encontro")
except Exception as e:
    print(f"Error procesando: {e}")

Problema 3:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


years = [2015, 2016, 2017, 2018, 2019, 2020]
try:
    import lxml
    life_cost = (
        pd.concat({
            year: (
                pd.read_html(f"https://www.numbeo.com/cost-of-living/rankings.jsp?title={year}")[1]
                .rename(columns=lambda x: x.lower().replace(" ", "_"))
                .assign(rank=lambda x: x.index + 1)
                .set_index("rank")
            ) for year in years
        })
        .rename_axis(["year", "rank"])
        .reset_index()
    )
    print(f"Datos cargados de shape {life_cost.shape}")

except ImportError:
    print("lxml not installed - installing now...")
    import subprocess
    subprocess.run(['pip', 'install', 'lxml', '-q'], capture_output=True)
    print("Please restart the script after installation")
    life_cost = None
    #Se instala lxml en caso de que el usuario no lo tenga
except Exception as e:
    print(f"No se logro conseguir los datos en linea. Verificar que existe una conexion: {e}")


if life_cost is not None:
    if 'cost_of_living_index' in life_cost.columns:
        fig, axes = plt.subplots(2, 3, figsize=(12, 8))
        axes = axes.flatten()

        for i, year in enumerate(years):
            year_data = life_cost[life_cost['year'] == year]['cost_of_living_index']
            if not year_data.empty:
                axes[i].hist(year_data, bins=20, edgecolor='black', alpha=0.7)
                axes[i].set_title(f'Year {year}')
                axes[i].set_xlabel('Cost of Living Index')
                axes[i].set_ylabel('Frecuencia')
                axes[i].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('cost_living_histograms.png')
        plt.close()
        print("Histogramas guardados!")
    else:
        print("Warning: columna cost_of_living_index no encontrada")


    if 'restaurant_price_index' in life_cost.columns:
        rol_seed = 202110551
        np.random.seed(rol_seed)

        unique_cities = life_cost['city'].unique()
        n_cities = min(10, len(unique_cities))
        my_cities = np.random.choice(unique_cities, size=n_cities, replace=False)

        plt.figure(figsize=(10, 6))
        for city in my_cities:
            city_data = life_cost[life_cost['city'] == city].sort_values('year')
            if not city_data.empty:
                plt.plot(city_data['year'], city_data['restaurant_price_index'],
                        marker='o', label=city[:15])  #Truncar nombres largos

        plt.xlabel('Year')
        plt.ylabel('Restaurant Price Index')
        plt.title('Evolucion del indice de Restaurante (10 Random Cities)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('restaurant_index_evolution.png')
        plt.close()
        print("Imagen saved!")
    else:
        print("Warning: restaurant_price_index column not found")
